In [1]:
from glob import glob
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import scipy
import sklearn
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split,KFold
import matplotlib.pyplot as plt
import seaborn as sns
import time
import operator
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,fbeta_score,make_scorer,mutual_info_score,silhouette_score,normalized_mutual_info_score,classification_report, confusion_matrix,f1_score, mean_squared_error, adjusted_rand_score
import time 
from NNnet_class import NNnet
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA,FastICA
from sklearn.random_projection import johnson_lindenstrauss_min_dim,SparseRandomProjection,GaussianRandomProjection
from sklearn.manifold import TSNE,trustworthiness,Isomap
from A3_utils import calculate_wcss,plot_gallery,categorizer,create_elbow_plot,experiment_em_clusters,experiment_km_clusters
from kmodes.kmodes import KModes
import umap
from matplotlib.offsetbox import (AnnotationBbox, DrawingArea, OffsetImage,
                                  TextArea)

In [2]:
import warnings
from scipy.sparse import SparseEfficiencyWarning
# Ignore SparseEfficiencyWarning
warnings.simplefilter('ignore', category=SparseEfficiencyWarning)

In [3]:
#Seeting seed
np.random.seed(42)

#Making accuracy scorrer
accuracy_scorer=make_scorer(accuracy_score)

#Global Beta (Used for Ftwo score - not used in assignment)
BETA=2

In [4]:
def create_param_curve(data,param,param_vals,ax,algorithim_name,algorithim,metric=accuracy_scorer,metric_name='Accuracy',graph=True,beta=BETA,cv=3):

    '''Function to create parameter curves
    
    Parameters:
    data (list): List of np.arrays containing the features and labels  
    param (str): Parameter to vary
    param_vals (list): Parameter values
    ax (matplotlib.axes): Axis for graph
    algorithim (sklearn.algorithim/NNnet): Algorithim to vary parameter in
    metric (sklearn.metric): Metric to score algorotihim on
    metric_name (str): Metric Name
    graph (bool): Bool for validation graph
    beta (int): Beta value for the Ftwo scored 
    cv (int): The number of cross validations folds
    
    Returns:
    None'''


    train_acc=[]
    test_acc=[]
    
    for i in param_vals:
        #Crossvalidating each parameter value
        kf=KFold(n_splits=cv,shuffle=True)

        internal_train_accuracy=0
        internal_test_accuracy=0
        
        for train,test in kf.split(X=data[0]):
            if i!='Default':
                if 'net' in algorithim_name.lower():
                    kwargs={param:i,'input_dims':data[0].shape[-1]}
                    clf=algorithim(**kwargs)
                else:
                    clf=algorithim(**{param:i})
            else:
                clf=algorithim()

            clf.fit(data[0][train],data[1][train])
            internal_train_accuracy+=metric(y_pred=clf.predict(data[0][train]),y_true=data[1][train])
            internal_test_accuracy+=metric(y_pred=clf.predict(data[0][test]),y_true=data[1][test])

        train_acc.append(internal_train_accuracy/cv)
        test_acc.append(internal_test_accuracy/cv)

    best_val=param_vals[np.argmax(test_acc)]

    if type(param_vals[0])==str:
        ax.scatter(param_vals,train_acc,label='Training {}'.format(metric_name))
        ax.scatter(param_vals,test_acc,label='Validation {}'.format(metric_name))
    else:
        ax.plot(param_vals,train_acc,label='Training {}'.format(metric_name))
        ax.plot(param_vals,test_acc,label='Validation {}'.format(metric_name))
    #plt.xscale('log')
    ax.axvline(best_val,label='Best {} Value'.format(param),color='red',linestyle = '--')
    ax.legend()
    ax.set_ylabel('{}'.format(metric_name));
    ax.set_xlabel(param);
    ax.set_title('{} vs {}'.format(param,metric_name));
    if type(best_val)!=str:
        if best_val==max(param_vals):
            ax.text(y=(max(test_acc)+min(test_acc))/2+np.abs(np.std(test_acc)),x=best_val-np.std(param_vals)/12,s=best_val,color='green',weight='bold')
        else:
            ax.text(y=(max(test_acc)+min(test_acc))/2+np.abs(np.std(test_acc)),x=best_val+np.std(param_vals)/12,s=best_val,color='green',weight='bold')

In [5]:
def deal_algorithim(data,param_dicts,dataset,algorithim_name,algorithim,metric=accuracy_score,metric_name='Accuracy',cv=3):
    
    '''Function to deal with algorithim
    
    Parameters:
    data (list): List of np.arrays containing the features and labels
    param_dict (dict): Parameter dictionary to vary
    dataset (str): Dataset name
    algorithim_name (str): Algorithim name
    algorithim (sklearn.algorithim/NNnet): Algorithim to vary parameter in
    metric (sklearn.metric): Metric to score algorotihim on
    metric_name (str): Metric Name
    cv (int): The number of cross validations folds 

    Returns:
    None'''
    
    num_classes=len(param_dicts.keys())

    #Getting Fig Size
    fig,axes=plt.subplots(2,int(np.ceil(num_classes/2)))
    fig.set_size_inches(15,15)
    i=-1
    for c,ax in enumerate(fig.axes):
        i+=1
        plt.suptitle('Results for Algorithim: "{}" for "{}" Dataset'.format(algorithim_name,dataset),fontsize=18)
        param=list(param_dicts.keys())[i]
        param_vals=param_dicts[param]
        create_param_curve(data,param,param_vals,ax,algorithim_name,algorithim,metric,metric_name)
    plt.tight_layout()

In [6]:
#Load Heart Disease Data
#Load Heart Disease Data
def load_heart_data():

    '''Load Heart Disease Dataset
    
    Returns:
    X (np.array): X array
    Y (np.array): Y array
    col_index (dict): Dictionary containing the pairing for the column location and it's name'''

    #PLEASE CHANGE TO LOCATION OF YOUR HEART DATA
    df=pd.read_csv('Data/Heart_2/heart.csv')
    Y=np.array(df['HeartDisease'])
    df.drop('HeartDisease',axis=1,inplace=True)
    
    label_columns=['HeartDisease']
    categorical_columns=['Sex', 'ChestPainType', 'RestingECG','ExerciseAngina','ST_Slope','FastingBS']

    non_categorical_variables=list(set(df.columns).difference(set(categorical_columns+label_columns)))
    X=np.array(df[non_categorical_variables])
    columns_categorized=non_categorical_variables

    #Now we need to one hot vectorize the type_of_meal_plan, room_type_reserved and market_segment_type
    label_dict={}
    for i in categorical_columns:
        label_dict[i]=OneHotEncoder()
        res=label_dict[i].fit_transform(np.array(df[i]).reshape(-1,1)).toarray()
        X=np.c_[X,res]
        columns_categorized=columns_categorized+[i+'%'+j for j in ['1','2','3','4','5','6','7'][:res.shape[-1]]]

        col_index={}
        results_corr={}
        for label,col in zip(columns_categorized,range(X.shape[-1])):
            corr=scipy.stats.pearsonr(X[:,col],Y)[0]
            results_corr[label]=corr
            col_index[label]=col
    return X,Y,col_index

#Fashion MNIST
def load_fmnist():
    #Loading dataset
    data=pd.concat([pd.read_csv('Data/FMNIST/fashion-mnist_train.csv'),pd.read_csv('Data/FMNIST/fashion-mnist_test.csv')],axis=0)

    X=data.iloc[:,1:].to_numpy()
    Y=data.iloc[:,:1].to_numpy().ravel()

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1,random_state=42,stratify=Y)

    return X_test,y_test


#Load Pokemon Data
def load_pokemon():
    directory='Data/Pokemon/'
    image_files = glob(os.path.join(directory, '**', '*.*'), recursive=True)

    # Initialize a list to store the images as NumPy arrays
    features=[]
    labels=[]

    # Iterate over the Image files
    for file in image_files:

        #Name of pokemon
        label=file.split('\\')[1]

        #Appending name
        labels.append(label)

        #Reading images
        image = Image.open(file)

        #Converting to RGB
        image = image.convert('RGB')

        #Removing color
        image=image.convert('L')
        
        #Resizing images
        image=image.resize((48, 48))

        #Appending images
        features.append(np.array(image).flatten())

    #Converting to Numpy
    labels=np.array(labels)
    features=np.array(features)

    pokemon_names=LabelEncoder()
    labels_encoded=pokemon_names.fit_transform(labels)

    return features,labels_encoded,pokemon_names

In [7]:
def score_algorithim(X,Y,dataset_name,algorithim_name,algorithim,params,predictor_metric=accuracy_score,grid_search_metric=accuracy_scorer,predictor_metric_name='accuracy',return_needed=False):

    '''Function to score algorithim
    
    Parameters:
    X (np.array): All features
    Y (np.array): All labels
    dataset_name (str): Dataset name
    algorithim_name (str): Algorithim name
    algorithim (sklearn.algorithim/NNnet): Algorithim being experimented on
    params (dict): Parameter dictionary for GridSearch
    predictor_metric (sklearn.metric): Metric to score algorotihim on
    grid_search_metric (sklearn.metric): Metric used by GridSearchCV
    predictor_metric_name (str): Metric Name for predictor metric
    return_needed (bool): Bool if fit model needed to be returned    

    Returns:
    glf (sklearn.model/NNnet): Fit model'''

    standardize=False
    if 'KNN' in algorithim_name.upper() or 'SVM' in algorithim_name.upper() or 'NET' in algorithim_name.upper():
        standardize=True
    
    if 'NET' in algorithim_name.upper():
        n_jobs=1
    else:
        n_jobs=-1


    train,test=split_data(X,Y,valid=False,standardize=standardize)

    glf_cv=GridSearchCV(algorithim,param_grid=params,verbose=0,n_jobs=n_jobs,cv=3,scoring=grid_search_metric,refit=True)

    rep=10
    start_time=time.time()
    glf_cv.fit(train[0],train[1])

    glf=glf_cv.best_estimator_
    
    for i in range(rep):
        glf.fit(train[0],train[1])
    time_delay_train=(time.time()-start_time)/rep

    y_train_pred=glf.predict(train[0])
    
    if 'F' in predictor_metric_name.upper():
        train_score = predictor_metric(y_pred=glf.predict(train[0]),y_true=train[1],beta=BETA)
        test_score = predictor_metric(y_pred=glf.predict(test[0]),y_true=test[1],beta=BETA)
    else:
        train_score = predictor_metric(y_pred=glf.predict(train[0]),y_true=train[1])
        test_score = predictor_metric(y_pred=glf.predict(test[0]),y_true=test[1])

    start_time=time.time()
    for i in range(rep):
        y_test_pred=glf.predict(test[0])
    time_delay_infer=(time.time()-start_time)/rep

    print('{} Final Results'.format(dataset_name))
    print('\n')
    print(glf_cv.best_params_)
    print('\n')
    print('Average time to train the ideal {} was {:.3f} seconds'.format(algorithim_name,time_delay_train))
    print('Average time to infer the ideal {} was {:.3f} seconds'.format(algorithim_name,time_delay_infer))
    print('\n')
    print('The result on the training data for the ideal {} algorithim is a {} {} score'.format(algorithim_name,train_score,predictor_metric_name))
    print('The result on the test data for the ideal {} algorithim is a {} {} score'.format(algorithim_name,test_score,predictor_metric_name))
    
    if return_needed:
        return glf

In [8]:
def split_data(X,Y,valid=True,standardize=False):

    '''
    Split the data between train, test and optional validation dataset

    Parameters:
    X (np.array): X features
    Y (np.rray): Labels
    valid (bool): Split into validation dataset 
    standardize (bool): Whether to standardize the data (introduces bias as Sklearn Standard Scaler is trained only on the train data)

    Returns:
    train (list): np.array list of train
    valid (list): optional np.array list of validation
    test (list): np.array list of test
    '''
    
    #Now let's split the data between test and train, we'll use the standard 80/20 split
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=42)
    
    if valid:
        #We'll also split the data between train and validation, we'll again use the standard 80/20 split
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2,random_state=42)
        
        if standardize:
            sklr=StandardScaler()
            X_train=sklr.fit_transform(X_train)
            X_valid=sklr.transform(X_valid)
            X_test=sklr.transform(X_test)
        return [X_train,y_train],[X_valid,y_valid],[X_test,y_test]

    if standardize:
        sklr=StandardScaler()
        X_train=sklr.fit_transform(X_train)
        X_test=sklr.transform(X_test)
    return [X_train,y_train],[X_test,y_test]

In [9]:
#Heart
X_heart,Y_heart,heart_index=load_heart_data()
train_heart,test_heart=split_data(X_heart,Y_heart,valid=False)

#Fashion MNIST
X_fm,Y_fm=load_fmnist()
train_fm,test_fm=split_data(X_fm,Y_fm,valid=False)

In [10]:
#Standardizing
sklr_poke=StandardScaler()
sklr_heart=StandardScaler()
sklr_fm=StandardScaler()

X_poke_scaler=StandardScaler()
X_heart_scaler=StandardScaler()
X_fm_scaler=StandardScaler()

#Heart
X_heart_scaled=X_heart_scaler.fit_transform(X_heart)

train_heart_standardized=train_heart.copy()

test_heart_standardized=test_heart.copy()

train_heart_standardized[0]=sklr_heart.fit_transform(train_heart[0])
test_heart_standardized[0]=sklr_heart.transform(test_heart[0])

#FMNIST
X_fm_scaled=X_fm_scaler.fit_transform(X_fm)

train_fm_standardized=train_fm.copy()
train_fm_unstandardized=test_fm.copy()

test_fm_standardized=test_fm.copy()

train_fm_standardized[0]=sklr_fm.fit_transform(train_fm[0])
test_fm_standardized[0]=sklr_fm.transform(test_fm[0])

# Clustering

### Expectation Maximization

#### FMNIST Data

In [11]:
#Testing EM algorithm with different number of components
n_list=np.arange(1,20)
dataset_name='FMNIST'
dataset=X_fm_scaled
labels_og=Y_fm
algorithm_name='EM'
cluster_graphs=4
repeats=3

#Converting dataset to 2D so it's plottable
converter=TSNE(n_components=2)
dataset_2d=converter.fit_transform(dataset)

#Average of 3 runs 
silhouette_scores=np.zeros(shape=len(n_list))
wcss_scores=np.zeros(shape=len(n_list))
BIC_scores=np.zeros(shape=len(n_list))
AIC_scores=np.zeros(shape=len(n_list))
EM_dict={}

for r in [11*(1+i) for i in range(repeats)]:

    #Will be calculated for each run for plots
    all_labels=[]
    
    #Assigning Clusters
    for i,n in enumerate(n_list):
        #Creating Gaussian
        em=GaussianMixture(n_components=n,random_state=r)

        #Fitted labels dataset
        em.fit(dataset)

        #Predictions
        labels=em.predict(dataset)
        all_labels.append(labels)

        #Calculating Silhouette Score
        if n==1: 
            pass
        else:
            silhouette_scores[i]=silhouette_score(dataset,labels)/repeats

        #WCSS Score
        wcss_scores[i]=calculate_wcss(dataset,labels,em.means_)/repeats

        #BIC Scores
        BIC_scores[i]=em.bic(dataset)/repeats

        #AIC
        AIC_scores[i]=em.aic(dataset)/repeats

        #Saving EM models for final Run 
        EM_dict[n]=em

    #Plotting 2d color coded with labels 
    fig,axes=plt.subplots(1,len(n_list[:cluster_graphs])+1)
    fig.set_size_inches(35,5)

    plt.suptitle(f'{dataset_name} dataset clusters using {algorithm_name}',fontsize=18)

    for c,ax in enumerate(fig.axes):

        if c>-1:
            #Predictions
            labels=all_labels[c+1]

            #Plotting
            sc=ax.scatter(dataset_2d[:,0],dataset_2d[:,1],c=labels,cmap='plasma')
            #ax.legend()
            ax.set_ylabel('Y');
            ax.set_xlabel('X');
            ax.set_title('{} : COMPONENTS'.format(n_list[c+1]));
            ax.legend(*sc.legend_elements(), title='clusters')

        else:

            sc=ax.scatter(dataset_2d[:,0],dataset_2d[:,1],c=labels_og,cmap='plasma')
            #ax.legend()
            ax.set_ylabel('Y');
            ax.set_xlabel('X');
            ax.set_title('Dataset');      
            ax.legend(*sc.legend_elements(), title='clusters')

    plt.tight_layout()

#Plotting Silhouette score
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('Silhouette_scores & Clusters')
plt.plot(n_list[1:],silhouette_scores[1:])
plt.scatter(n_list[1:], silhouette_scores[1:], s=100, c='blue', marker='o', edgecolors='black')
plt.xticks(n_list[1:])
plt.xlabel('N_clusters')
plt.ylabel('Silhouette_scores')

plt.tight_layout()

#Plotting WCSS
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('WCSS & Clusters')
plt.plot(n_list,wcss_scores)
plt.scatter(n_list, wcss_scores, s=100, c='blue', marker='o', edgecolors='black')
plt.ylabel('WCSS')
plt.xticks(n_list)
plt.xlabel('N_clusters')
ax_2=plt.gca().twinx()
create_elbow_plot(n_list,wcss_scores,ax_2)
plt.legend()

plt.tight_layout()

#Plotting BIC
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('BIC Score & Clusters')
plt.plot(n_list,BIC_scores)
plt.scatter(n_list, BIC_scores, s=100, c='blue', marker='o', edgecolors='black')
plt.xticks(n_list)
plt.xlabel('N_clusters')
plt.ylabel('BIC Score')

#Plotting AIC
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('AIC Score & Clusters')
plt.plot(n_list,AIC_scores)
plt.scatter(n_list, AIC_scores, s=100, c='blue', marker='o', edgecolors='black')
plt.ylabel('AIC Score')
plt.xticks(n_list)
plt.xlabel('N_clusters')
ax_2=plt.gca().twinx()
create_elbow_plot(n_list,AIC_scores,ax_2)
plt.legend()

plt.tight_layout()

In [ ]:
#Best Number of Clusters
#Choosing 5 as inspecting graphs + domain knowledge indicates 7 is closer to reality 
best_clusters_d1=8

#Plotting the Mean Cluster Image from the ideal number of customers
#Best EM model
best_em_d1=GaussianMixture(n_components=best_clusters_d1,random_state=21)
best_em_d1.fit(X_fm_scaled)
#Plotting the best cluster means

titles=[f'Centroid {i+1}' for i in range(len(best_em_d1.means_))]
centroids_viz =best_em_d1.means_.reshape(best_clusters_d1,28,28)
plot_gallery(centroids_viz,titles, 28, 28,n_row=1,n_col=8)

In [ ]:
d1_dict_em=EM_dict.copy()

In [ ]:
for i in range(5):
    #Evaluation Normalized MI Score
    best_em_d1=GaussianMixture(n_components=best_clusters_d1,random_state=21)
    mi_score=normalized_mutual_info_score(labels_og,best_em_d1.predict(dataset))
    print(f'The Normalized Mutual Info Score for the best clustering is {mi_score}') 

    #Evaluating Adjusted Rand Score Rand Score
    aj_score=adjusted_rand_score(labels_og,best_em_d1.predict(dataset))
    print(f'The Adjusted Rand Score for the best clustering is {aj_score}') 

In [ ]:
#Visualizing
tsne_vis=TSNE(n_components=2,perplexity=45,n_jobs=-1)

#Transforming dataset 1 based on ideal 
dataset_transformed=tsne_vis.fit_transform(dataset)

fig=plt.figure()
fig, axes = plt.subplots(1,2,figsize=(15, 5))

position_dict={}

plt.suptitle('Visualizations EM Clustering Results with Labels (FMINST)',fontsize=18)
LABS=[labels_og,best_em_d1.predict(dataset)]

for lab in np.unique(labels_og):
    x,y=dataset_transformed[labels_og==lab][:,0].mean(),dataset_transformed[labels_og==lab][:,1].mean()
    position_dict[lab]=[x,y]

axis_int=0
for ax,labels_ in zip(axes,LABS):

    artists=[]
    sc=ax.scatter(dataset_transformed[:,0],dataset_transformed[:,1],c=labels_,cmap='viridis',zorder=1)
    # Overlay each image within the scatter plot

    for lab in np.unique(labels_og):
        # Positioning Images
        image_x = position_dict[lab][0]
        image_y = position_dict[lab][1]
        im=OffsetImage(dataset[labels_og==lab][0].reshape(28,28),cmap='gray')
        ab = AnnotationBbox(im, (image_x, image_y), xycoords='data', frameon=False)
        artists.append(ax.add_artist(ab))

    if axis_int==0:
        ax.set_title(f'Actual')
        axis_int+=1
    else:
        ax.set_title(f'Clustered')

    ax.legend(*sc.legend_elements(), title='Label')
plt.tight_layout()

#### Heart Data

In [ ]:
#Testing EM algorithm with different number of components
n_list=np.arange(1,20)
dataset_name='Heart Disease'
dataset=X_heart_scaled
labels_og=Y_heart
algorithm_name='EM'
cluster_graphs=4
repeats=3

#Converting dataset to 2D so it's plottable
converter=TSNE(n_components=2)
dataset_2d=converter.fit_transform(dataset)

#Average of 3 runs 
silhouette_scores=np.zeros(shape=len(n_list))
wcss_scores=np.zeros(shape=len(n_list))
BIC_scores=np.zeros(shape=len(n_list))
AIC_scores=np.zeros(shape=len(n_list))
EM_dict={}

for r in [21*(1+i) for i in range(repeats)]:

    #Will be calculated for each run for plots
    all_labels=[]
    
    #Assigning Clusters
    for i,n in enumerate(n_list):
        #Creating Gaussian
        em=GaussianMixture(n_components=n,random_state=r)

        #Fitted labels dataset
        em.fit(dataset)

        #Predictions
        labels=em.predict(dataset)
        all_labels.append(labels)

        #Calculating Silhouette Score
        if n==1:
            pass
        else:
            silhouette_scores[i]=silhouette_score(dataset,labels)/repeats

        #WCSS Score
        wcss_scores[i]=calculate_wcss(dataset,labels,em.means_)/repeats

        #BIC Scores
        BIC_scores[i]=em.bic(dataset)/repeats

        #AIC
        AIC_scores[i]=em.aic(dataset)/repeats

        #Saving EM models for final Run 
        EM_dict[n]=em

    #Plotting 2d color coded with labels 
    fig,axes=plt.subplots(1,len(n_list[:cluster_graphs])+1)
    fig.set_size_inches(35,5)

    plt.suptitle(f'{dataset_name} dataset clusters using {algorithm_name}',fontsize=18)

    for c,ax in enumerate(fig.axes):

        if c>-1:
            #Predictions
            labels=all_labels[c+1]

            #Plotting
            sc=ax.scatter(dataset_2d[:,0],dataset_2d[:,1],c=labels,cmap='plasma')
            #ax.legend()
            ax.set_ylabel('Y');
            ax.set_xlabel('X');
            ax.set_title('{} : COMPONENTS'.format(n_list[c+1]));
            ax.legend(*sc.legend_elements(), title='clusters')

        else:

            sc=ax.scatter(dataset_2d[:,0],dataset_2d[:,1],c=labels_og,cmap='plasma')
            #ax.legend()
            ax.set_ylabel('Y');
            ax.set_xlabel('X');
            ax.set_title('Dataset');      
            ax.legend(*sc.legend_elements(), title='clusters')

    plt.tight_layout()

#Plotting Silhouette score
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('Silhouette_scores & Clusters')
plt.plot(n_list[1:],silhouette_scores[1:])
plt.scatter(n_list[1:],silhouette_scores[1:], s=100, c='blue', marker='o', edgecolors='black')
plt.xticks(n_list[1:])
plt.xlabel('N_clusters')
plt.ylabel('Silhouette_scores')

plt.tight_layout()

#Plotting WCSS
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('WCSS & Clusters')
plt.plot(n_list,wcss_scores)
plt.scatter(n_list, wcss_scores, s=100, c='blue', marker='o', edgecolors='black')
plt.ylabel('WCSS')
plt.xticks(n_list)
plt.xlabel('N_clusters')
ax_2=plt.gca().twinx()
create_elbow_plot(n_list,wcss_scores,ax_2)
plt.legend()

plt.tight_layout()

#Plotting BIC
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('BIC Score & Clusters')
plt.plot(n_list,BIC_scores)
plt.scatter(n_list, BIC_scores, s=100, c='blue', marker='o', edgecolors='black')
plt.xticks(n_list)
plt.xlabel('N_clusters')
plt.ylabel('BIC Score')

#Plotting AIC
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('AIC Score & Clusters')
plt.plot(n_list,AIC_scores)
plt.scatter(n_list, AIC_scores, s=100, c='blue', marker='o', edgecolors='black')
plt.ylabel('AIC Score')
plt.xticks(n_list)
plt.xlabel('N_clusters')
ax_2=plt.gca().twinx()
create_elbow_plot(n_list,AIC_scores,ax_2)
plt.legend()

plt.tight_layout()

In [ ]:
#Selecting the best number of cluster and plotting pair plots 
dataset=X_heart_scaled
#Best Number of Clusters
best_clusters_em_d2=7

for i in range(5):
    #Best EM model
    best_em_d2=GaussianMixture(n_components=best_clusters_em_d2,random_state=63)

    best_em_d2.fit(dataset)
    #Getting labels for best cluster
    labels=best_em_d2.predict(dataset)

    #Evaluation Normalized MI Score
    mi_score=normalized_mutual_info_score(labels_og,best_em_d2.predict(dataset))
    print(f'The Normalized Mutual Info Score for the best clustering iS {mi_score}') 

    #Evaluating Adjusted Rand Score Rand Score
    aj_score=adjusted_rand_score(labels_og,best_em_d2.predict(dataset))
    print(f'The Adjusted Rand Score for the best clustering iS {aj_score}') 

In [ ]:
#Plotting Result
#Visualizing
tsne_vis=TSNE(n_components=2,perplexity=45,n_jobs=-1)

#Transforming dataset 1 based on ideal 
dataset_transformed=tsne_vis.fit_transform(dataset)

fig=plt.figure()
fig, axes = plt.subplots(1,2,figsize=(15, 5))

position_dict={}

plt.suptitle('Visualizations EM Clustering (HEART)',fontsize=18)
LABS=[labels_og,best_em_d2.predict(dataset)]

axis_int=0
for ax,labels_ in zip(axes,LABS):

    artists=[]
    sc=ax.scatter(dataset_transformed[:,0],dataset_transformed[:,1],c=labels_,cmap='viridis',zorder=1)
    # Overlay each image within the scatter plot

    if axis_int==0:
        ax.set_title(f'Actual')
        axis_int+=1
    else:
        ax.set_title(f'Clustered')

    ax.legend(*sc.legend_elements(), title='Label')
plt.tight_layout()

In [ ]:
ROWS=2
#Plotting with actual labels

TWO_FIXED=['Oldpeak','MaxHR']
ind_1=heart_index[TWO_FIXED[0]]
ind_2=heart_index[TWO_FIXED[1]]

filter_cols=[i for i in range(len(heart_index)) if i not in (ind_1,ind_2)]
filtered_heart=X_heart_scaled[:,filter_cols]
reversed_dict={i:k for k,i in heart_index.items()}

fig,axes=plt.subplots(ROWS,4,subplot_kw={'projection': '3d'})
fig.set_size_inches(ROWS*(len(reversed_dict)-4)/ROWS,5*ROWS)
plt.suptitle(f'Visualizing Best Clusters By Feature',fontsize=18)

for c,ax in enumerate(fig.axes):

    if c%2==0:
        #Plot Actual With Binary
        sc=ax.scatter(X_heart_scaled[:,ind_1],X_heart_scaled[:,ind_2],filtered_heart[:,c],c=labels_og,cmap='viridis',alpha=0.75,s=10)
        ax.set_xlabel(f'{TWO_FIXED[0]}',labelpad=-13)
        ax.set_ylabel(f'{TWO_FIXED[1]}',labelpad=-13)
        ax.set_zlabel(f'{reversed_dict[filter_cols[c]]}',labelpad=-13)
        ax.set_title(f'Actual Clusters for {reversed_dict[filter_cols[c]]}')
        ax.legend(*sc.legend_elements(), title='clusters')
        
        # Remove fill
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False
        ax.xaxis.pane.set_edgecolor('w')
        ax.yaxis.pane.set_edgecolor('w')
        ax.zaxis.pane.set_edgecolor('w')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])
        ax.grid(False)

        #Plot Predicted Cluster
        sc=fig.axes[c+1].scatter(X_heart_scaled[:,ind_1],X_heart_scaled[:,ind_2],filtered_heart[:,c],c=labels,cmap='viridis',alpha=0.75,s=10)
        fig.axes[c+1].set_xlabel(f'{TWO_FIXED[0]}',labelpad=-13)
        fig.axes[c+1].set_ylabel(f'{TWO_FIXED[1]}',labelpad=-13)
        fig.axes[c+1].set_zlabel(f'{reversed_dict[filter_cols[c]]}',labelpad=-13)
        fig.axes[c+1].set_title(f'Predicted Clusters for {reversed_dict[filter_cols[c]]}')
        fig.axes[c+1].legend(*sc.legend_elements(), title='clusters')
        
        # Remove fill
        fig.axes[c+1].xaxis.pane.fill = False
        fig.axes[c+1].yaxis.pane.fill = False
        fig.axes[c+1].zaxis.pane.fill = False
        fig.axes[c+1].xaxis.pane.set_edgecolor('w')
        fig.axes[c+1].yaxis.pane.set_edgecolor('w')
        fig.axes[c+1].zaxis.pane.set_edgecolor('w')
        fig.axes[c+1].set_xticks([])
        fig.axes[c+1].set_yticks([])
        fig.axes[c+1].set_zticks([])
        fig.axes[c+1].grid(False)

    else:
        continue

plt.tight_layout()
plt.subplots_adjust(top=0.95)

In [ ]:
d2_dict_em=EM_dict.copy()

### K Modes

#### FMNIST Data

In [ ]:
#Testing K_Modes algorithm with different number of components
dataset_name='FMNIST'
dataset=X_fm_scaled
labels_og=Y_fm
algorithm_name='K-MODES'
categorical_cols=None

experiment_km_clusters(dataset=dataset,dataset_name=dataset_name,labels_og=labels_og,algorithm_name=algorithm_name,categorical_cols=categorical_cols)

In [ ]:
#Evaluation
dataset=X_fm_scaled
best_clusters=2
categorical_cols=None
data_categorized=categorizer(dataset,categorical_cols)
labels_og=Y_fm
labels=labels_og

for i in range(5):
    #Kmodes
    best_km_d1 = KModes(n_clusters=best_clusters, init='Huang', n_init=3, verbose=0)

    #Best KM model
    best_km_d1.fit(data_categorized,categorical=np.arange(data_categorized.shape[-1]))
    labels=best_km_d1.predict(data_categorized,categorical=np.arange(data_categorized.shape[-1]))

    print("DATASET 1 KMODES RESULTS \n\n")

    #Evaluation Normalized MI Score
    mi_score=normalized_mutual_info_score(labels_og,best_km_d1.predict(data_categorized,categorical=np.arange(data_categorized.shape[-1])))
    print(f'The Normalized Mutual Info Score for the best clustering iS {mi_score}') 

    #Evaluating Adjusted Rand Score Rand Score
    aj_score=adjusted_rand_score(labels_og,best_km_d1.predict(data_categorized,categorical=np.arange(data_categorized.shape[-1])))
    print(f'The Adjusted Rand Score for the best clustering iS {aj_score}') 

In [ ]:
#Plotting the Mean Cluster Image from the ideal number of clusters 

#Visualizing Results from KFOLDS
tsne_vis=TSNE(n_components=2,perplexity=30,n_jobs=-1)

#Transforming dataset 1 based on ideal 
dataset_transformed=tsne_vis.fit_transform(data_categorized)

fig=plt.figure()
fig, axes = plt.subplots(1,2,figsize=(25, 5))

position_dict={}

plt.suptitle('Visualizations K-MODES Clustering Results with Labels (FMINST)',fontsize=18)
LABS=[labels_og,best_km_d1.predict(data_categorized,categorical=data_categorized.shape[-1])]

for lab in np.unique(labels_og):
    x, y = dataset_transformed[labels_og == lab][:, 0].mean(), dataset_transformed[labels_og == lab][:, 1].mean()
    position_dict[lab] = [x, y]

axis_int = 0
for ax, labels_ in zip(axes, LABS):
    artists = []

    for l in np.unique(labels_):
        # Choose color from a colormap
        color = plt.cm.viridis(l / len(np.unique(labels_)))
        ax.scatter(dataset_transformed[:, 0][labels_ == l], dataset_transformed[:, 1][labels_ == l],
                   label=l, zorder=1, color=color)

    # Overlay each image within the scatter plot
    for lab in np.unique(labels_og):
        # Positioning Images
        image_x = position_dict[lab][0]
        image_y = position_dict[lab][1]
        im = OffsetImage(X_fm_scaled[labels_og == lab][0].reshape(28, 28), cmap='gray')
        ab = AnnotationBbox(im, (image_x, image_y), xycoords='data', frameon=False)
        artists.append(ax.add_artist(ab))

    if axis_int == 0:
        ax.set_title(f'Actual')
        axis_int += 1
    else:
        ax.set_title(f'Clustered')

    ax.legend()

#plt.tight_layout()
plt.show()

In [ ]:
#Plotting the Mean Cluster Image from the ideal number of clusters 

#Visualizing Results from KFOLDS
tsne_vis=TSNE(n_components=2,perplexity=30,n_jobs=-1)

#Transforming dataset 1 based on ideal 
dataset_transformed=tsne_vis.fit_transform(data_categorized)

fig=plt.figure()
fig, axes = plt.subplots(1,2,figsize=(25, 5))

position_dict={}

plt.suptitle('Visualizations K-MODES Clustering Results with Labels (FMINST) - CATEGORIZED',fontsize=18)
LABS=[labels_og,best_km_d1.predict(data_categorized,categorical=data_categorized.shape[-1])]

for lab in np.unique(labels_og):
    x, y = dataset_transformed[labels_og == lab][:, 0].mean(), dataset_transformed[labels_og == lab][:, 1].mean()
    position_dict[lab] = [x, y]

axis_int = 0
for ax, labels_ in zip(axes, LABS):
    artists = []

    for l in np.unique(labels_):
        # Choose color from a colormap
        color = plt.cm.viridis(l / len(np.unique(labels_)))
        ax.scatter(dataset_transformed[:, 0][labels_ == l], dataset_transformed[:, 1][labels_ == l],
                   label=l, zorder=1, color=color)

    # Overlay each image within the scatter plot
    for lab in np.unique(labels_og):
        # Positioning Images
        image_x = position_dict[lab][0]
        image_y = position_dict[lab][1]
        im = OffsetImage(X_fm_scaled[labels_og == lab][0].reshape(28, 28), cmap='gray')
        ab = AnnotationBbox(im, (image_x, image_y), xycoords='data', frameon=False)
        artists.append(ax.add_artist(ab))

    if axis_int == 0:
        ax.set_title(f'Actual')
        axis_int += 1
    else:
        ax.set_title(f'Clustered')

    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
#Plotting the Mean Cluster Image from the ideal number of clusters 
#Visualizing Results from KFOLDS
tsne_vis=TSNE(n_components=2,perplexity=45,n_jobs=-1)

#Transforming dataset 1 based on ideal 
dataset_transformed=tsne_vis.fit_transform(X_fm_scaled)

fig, axes = plt.subplots(1, 2, figsize=(25, 5))

position_dict = {}

plt.suptitle('Visualizations K-MODES Clustering Results with Labels (FMINST)', fontsize=18)
LABS=[labels_og,best_km_d1.predict(data_categorized,categorical=data_categorized.shape[-1])]

for lab in np.unique(labels_og):
    x, y = dataset_transformed[labels_og == lab][:, 0].mean(), dataset_transformed[labels_og == lab][:, 1].mean()
    position_dict[lab] = [x, y]

axis_int = 0
for ax, labels_ in zip(axes, LABS):
    artists = []

    for l in np.unique(labels_):
        # Choose color from a colormap
        color = plt.cm.viridis(l / len(np.unique(labels_)))
        ax.scatter(dataset_transformed[:, 0][labels_ == l], dataset_transformed[:, 1][labels_ == l],
                   label=l, zorder=1, color=color)

    # Overlay each image within the scatter plot
    for lab in np.unique(labels_og):
        # Positioning Images
        image_x = position_dict[lab][0]
        image_y = position_dict[lab][1]
        im = OffsetImage(X_fm_scaled[labels_og == lab][0].reshape(28, 28), cmap='gray')
        ab = AnnotationBbox(im, (image_x, image_y), xycoords='data', frameon=False)
        artists.append(ax.add_artist(ab))

    if axis_int == 0:
        ax.set_title(f'Actual')
        axis_int += 1
    else:
        ax.set_title(f'Clustered')

    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
d1_dict_km=KM_dict.copy()

#### Heart Data

In [ ]:
#Testing K_Modes algorithm with different number of components
n_list=np.arange(1,20)
dataset_name='Heart Disease'
dataset=X_heart_scaled
labels_og=Y_heart
algorithm_name='K-MODES'
cluster_graphs=4
categorical=[i for i,k in enumerate(heart_index.keys()) if '%' in k]

data_categorized=categorizer(dataset,categorical)

#Converting dataset to 2D so it's plottable
converter=TSNE(n_components=2)
dataset_2d=converter.fit_transform(data_categorized)
silhouette_scores=[]
all_labels=[]
wcss_scores=[]
KM_cost=[]
KM_dict={}


#Assigning Clusters
for n in tqdm(n_list):
    #Creating KMODS
    km = KModes(n_clusters=n, init='Huang', n_init=5, verbose=0)

    #Fitted labels dataset
    km.fit(data_categorized,categorical=np.arange(data_categorized.shape[-1]))

    #Predictions
    labels=km.predict(data_categorized,categorical=np.arange(data_categorized.shape[-1]))
    all_labels.append(labels)
    
    #silhouette score
    #Calculating Silhouette Score
    if n==1:
        pass
    else:
        silhouette_scores.append(silhouette_score(data_categorized,labels))
        
        
    #WCSS Score
    wcss_scores.append(calculate_wcss(data_categorized,labels,km.cluster_centroids_))

    KM_dict[n]=km

    KM_cost.append(km.cost_)

#Plotting 2d color coded with labels 
fig,axes=plt.subplots(1,len(n_list[:cluster_graphs])+1)
fig.set_size_inches(35,5)

plt.suptitle(f'{dataset_name} dataset clusters using {algorithm_name}',fontsize=18)

for c,ax in enumerate(fig.axes):

    if c>-1:
        #Predictions
        labels=all_labels[c+1]

        #Plotting
        sc=ax.scatter(dataset_2d[:,0],dataset_2d[:,1],c=labels,cmap='plasma')
        #ax.legend()
        ax.set_ylabel('Y');
        ax.set_xlabel('X');
        ax.set_title('{} : COMPONENTS'.format(n_list[c+1]));
        ax.legend(*sc.legend_elements(), title='clusters')

    else:

        sc=ax.scatter(dataset_2d[:,0],dataset_2d[:,1],c=labels_og,cmap='plasma')
        #ax.legend()
        ax.set_ylabel('Y');
        ax.set_xlabel('X');
        ax.set_title('Dataset');      
        ax.legend(*sc.legend_elements(), title='clusters')

plt.tight_layout()

#Plotting WCSS
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('WCSS & Clusters')
plt.plot(n_list,wcss_scores)
plt.scatter(n_list, wcss_scores, s=100, c='blue', marker='o', edgecolors='black')
plt.xticks(n_list)
ax_2=plt.gca().twinx()
create_elbow_plot(n_list,wcss_scores,ax_2)
plt.xlabel('N_clusters')
plt.ylabel('WCSS')
plt.legend()

plt.tight_layout()

#Plotting Cost
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('Cost & Clusters')
plt.plot(n_list,KM_cost)
plt.scatter(n_list, KM_cost, s=100, c='blue', marker='o', edgecolors='black')
plt.xticks(n_list)
ax_2=plt.gca().twinx()
create_elbow_plot(n_list,KM_cost,ax_2)
plt.xlabel('N_clusters')
plt.ylabel('Cost')
plt.legend()

plt.tight_layout()

#Plotting Silhouette score
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('Silhouette_scores & Clusters')
plt.plot(n_list[1:],silhouette_scores)
plt.scatter(n_list[1:], silhouette_scores, s=100, c='blue', marker='o', edgecolors='black')
plt.xticks(n_list[1:])
plt.xlabel('N_clusters')
plt.ylabel('Silhouette_scores')

plt.tight_layout()

In [ ]:
#Evaluation Results
labels_og=Y_heart

categorical=[i for i,k in enumerate(heart_index.keys()) if '%' in k]
data_categorized=categorizer(X_heart_scaled,categorical)

dataset=data_categorized


REPEATS=5

#Selecting N based on Plots
best_clusters=2
best_km_d2=KModes(n_clusters=best_clusters, init='Huang', n_init=5, verbose=0)

best_km_d2.fit(dataset,categorical=np.arange(dataset.shape[-1]))
#best_em_d1=EM_dict[best_clusters_PCA_d1]

print("DATASET 2 KMODES RESULTS \n\n")

#Evaluation Normalized MI & Adjusted Rand Score Score

mi_score=0
aj_score=0

for i in range(REPEATS):
    mi_score+=normalized_mutual_info_score(labels_og,best_km_d2.predict(dataset,categorical=np.arange(dataset.shape[-1])))/REPEATS
    aj_score+=adjusted_rand_score(labels_og,best_km_d2.predict(dataset,categorical=np.arange(dataset.shape[-1])))/REPEATS

print(f'The Normalized Mutual Info Score for the best clustering iS {mi_score}') 
print(f'The Adjusted Rand Score for the best clustering iS {aj_score}') 

In [ ]:
TWO_FIXED=['Oldpeak','MaxHR']
LABELS=labels_og
dataset=data_categorized
ind_1=heart_index[TWO_FIXED[0]]
ind_2=heart_index[TWO_FIXED[1]]
pred_labels=best_km_d2.predict(data_categorized,categorical=np.arange(data_categorized.shape[-1]))


filter_cols=[i for i in range(len(heart_index)) if i not in (ind_1,ind_2)]
filtered_heart=X_heart_scaled[:,filter_cols]
reversed_dict={i:k for k,i in heart_index.items()}

fig,axes=plt.subplots(ROWS,4,subplot_kw={'projection': '3d'})
fig.set_size_inches(ROWS*(len(reversed_dict)-4)/ROWS,5*ROWS)
plt.suptitle(f'Visualizing K-MODES Clustering Results By Features (ISOMAP HEART)',fontsize=18)

for c,ax in enumerate(fig.axes):

    if c%2==0:
        #Plot Actual With Binary
        for lab in np.unique(LABELS):
            color = plt.cm.viridis(lab / len(np.unique(LABELS)))
            ax.scatter(dataset[:,ind_1][LABELS==lab],dataset[:,ind_2][LABELS==lab],filtered_heart[:,c][LABELS==lab],label=lab,zorder=1,color=color,alpha=0.75,s=10)
        ax.set_xlabel(f'{TWO_FIXED[0]}',labelpad=-13)
        ax.set_ylabel(f'{TWO_FIXED[1]}',labelpad=-13)
        ax.set_zlabel(f'{reversed_dict[filter_cols[c]]}',labelpad=-13)
        ax.set_title(f'Actual Clusters for {list(heart_index.keys())[filter_cols[c]]}')
        ax.legend()
        
        # Remove fill
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False
        ax.xaxis.pane.set_edgecolor('w')
        ax.yaxis.pane.set_edgecolor('w')
        ax.zaxis.pane.set_edgecolor('w')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])
        ax.grid(False)

        #Plot Predicted Cluster
        for lab in np.unique(pred_labels):
            color = plt.cm.viridis(lab / len(np.unique(pred_labels)))
            fig.axes[c+1].scatter(dataset[:,ind_1][pred_labels==lab],dataset[:,ind_2][pred_labels==lab],filtered_heart[:,c][pred_labels==lab],label=lab,zorder=1,color=color,alpha=0.75,s=10)
        fig.axes[c+1].set_xlabel(f'{TWO_FIXED[0]}',labelpad=-13)
        fig.axes[c+1].set_ylabel(f'{TWO_FIXED[1]}',labelpad=-13)
        fig.axes[c+1].set_zlabel(f'{reversed_dict[filter_cols[c]]}',labelpad=-13)
        fig.axes[c+1].set_title(f'Predicted Clusters for {list(heart_index.keys())[filter_cols[c]]}')
        fig.axes[c+1].legend()
        
        # Remove fill
        fig.axes[c+1].xaxis.pane.fill = False
        fig.axes[c+1].yaxis.pane.fill = False
        fig.axes[c+1].zaxis.pane.fill = False
        fig.axes[c+1].xaxis.pane.set_edgecolor('w')
        fig.axes[c+1].yaxis.pane.set_edgecolor('w')
        fig.axes[c+1].zaxis.pane.set_edgecolor('w')
        fig.axes[c+1].set_xticks([])
        fig.axes[c+1].set_yticks([])
        fig.axes[c+1].set_zticks([])
        fig.axes[c+1].grid(False)

    else:
        continue

plt.tight_layout()
plt.subplots_adjust(top=0.95)

In [ ]:
df = pd.DataFrame(dataset, columns=heart_index)
df['Cluster']=labels

sns.pairplot(df, hue='Cluster', palette='viridis', diag_kind='kde')
plt.suptitle('Pair Plot of Best Clusters By Feature', fontsize=18)
plt.show()

# Dimensionality Reduction

### PCA

#### FMINST Data

In [ ]:
#Applying PCA on the FMINST Dataset

#Remembering that the number of features for FMINST are 784
dataset=X_fm_scaled
chosen_n_pca_d1=None

pca_size=range(250)
repeats=3
ideal_comps=None 
LIMIT=0.875

explained_variance=np.zeros(len(pca_size))
reconstruction_error=np.zeros(len(pca_size))

for r in [21*(1+i) for i in range(repeats)]:

    for i_ind,i in enumerate(pca_size):
        p_comp=PCA(n_components=i,random_state=r)
        p_comp.fit(dataset)
        explained_variance[i_ind]+=sum(p_comp.explained_variance_ratio_)/repeats

        #Calcualting Reconstruction Error
        dataset_transformed=p_comp.transform(dataset)
        dataset_reconstructed=p_comp.inverse_transform(dataset_transformed)
        reconstruction_error[i_ind]+=mean_squared_error(dataset,dataset_reconstructed)/repeats

#Plotting explained variance 
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('Explained Variance & Number of PCA Components (FMINST)')
plt.plot(pca_size,explained_variance)
plt.scatter(pca_size, explained_variance, s=10, c='blue', marker='o', edgecolors='black')
plt.axhline(y=LIMIT,color='red',linestyle='--')
plt.xlabel('N_components')
plt.ylabel('Explained Variance')

#Plotting reconstruction error 
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('Reconstruction Error & Number of PCA Components (FMINST)')
plt.plot(pca_size,reconstruction_error)
plt.scatter(pca_size, reconstruction_error, s=10, c='blue', marker='o', edgecolors='black')
plt.axvline(np.argwhere(explained_variance>LIMIT)[0][0],color='red',linestyle='--',label=f'chosen Components')
plt.xlabel('N_components')
plt.ylabel('Reconstruction Error')
plt.legend()

print('The lowest number of componenets to exceed the limit of {} is {}'.format(LIMIT,np.argwhere(explained_variance>LIMIT)[0][0]))
chosen_n_pca_d1=np.argwhere(explained_variance>LIMIT)[0][0]

##### Visualizing PCA Results for FMNIST

In [ ]:
#Plotting the images for the componenets chosen
chosen_n_pca_d1=75
dataset=X_fm_scaled
p_comp=PCA(n_components=chosen_n_pca_d1)
p_comp.fit(dataset)
titles=[f'Eigenvalue {i+1}' for i in range(chosen_n_pca_d1)]
eigenfashion = p_comp.components_.reshape(chosen_n_pca_d1,28,28)
plot_gallery(eigenfashion,titles, 28, 28,n_row=1,n_col=8)

#### Heart Data

In [ ]:
#Applying PCA on the FMINST Dataset

#Remembering that the number of features for FMINST are 784
dataset=X_heart_scaled
chosen_n_pca_d2=None

pca_size=range(20)
repeats=3
ideal_comps=None 
LIMIT=0.85

explained_variance=np.zeros(len(pca_size))
reconstruction_error=np.zeros(len(pca_size))

for r in [21*(1+i) for i in range(repeats)]:

    for i_ind,i in enumerate(pca_size):
        p_comp=PCA(n_components=i,random_state=r)
        p_comp.fit(dataset)
        explained_variance[i_ind]+=sum(p_comp.explained_variance_ratio_)/repeats

        #Calcualting Reconstruction Error
        dataset_transformed=p_comp.transform(dataset)
        dataset_reconstructed=p_comp.inverse_transform(dataset_transformed)
        reconstruction_error[i_ind]+=mean_squared_error(dataset,dataset_reconstructed)/repeats

#Plotting explained variance 
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('Explained Variance & Number of PCA Components (HEART)')
plt.plot(pca_size,explained_variance)
plt.scatter(pca_size, explained_variance, s=10, c='blue', marker='o', edgecolors='black')
plt.axhline(y=LIMIT,color='red',linestyle='--')
plt.xlabel('N_components')
plt.ylabel('Explained Variance')

#Plotting reconstruction error 
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('Reconstruction Error & Number of PCA Components (HEART)')
plt.plot(pca_size,reconstruction_error)
plt.scatter(pca_size, reconstruction_error, s=10, c='blue', marker='o', edgecolors='black')
plt.axvline(np.argwhere(explained_variance>LIMIT)[0][0],color='red',linestyle='--',label=f'chosen Components')
plt.xlabel('N_components')
plt.ylabel('Reconstruction Error')
plt.legend()

print('The lowest number of components to exceed the limit of {} is {}'.format(LIMIT,np.argwhere(explained_variance>LIMIT)[0][0]))
chosen_n_pca_d2=np.argwhere(explained_variance>LIMIT)[0][0]

##### Visualizing PCA Results for FMNIST

In [ ]:
#Plotting the images for the componenets chosen
dataset=X_heart_scaled
labels=Y_heart
p_comp=PCA(n_components=chosen_n_pca_d2)
p_comp.fit(dataset)
PLOT_FACTOR=5

#Projecting the dataset into the first 2 dimensions to identify most important features
fig=plt.figure()
fig.set_size_inches(10,5)
dataset_2d=np.dot(dataset,p_comp.components_[:2,:].T)

plt.title('Visualizing PCA & Most Important Features for Heart Dataset')
sc=plt.scatter(dataset_2d[:,0],dataset_2d[:,1],c=labels,cmap='plasma',s=5)
most_important=np.argsort(np.abs(p_comp.components_[:2]).sum(axis=0))[::-1][:4].ravel()

for i in range(dataset.shape[-1]):
    if i in most_important:
        plt.arrow(0,0,p_comp.components_[:2,i].T[0]*PLOT_FACTOR,p_comp.components_[:2,i].T[1]*PLOT_FACTOR,color='r',alpha=0.5)
        plt.text(p_comp.components_[:2,i].T[0]*PLOT_FACTOR*1.15,p_comp.components_[:2,i].T[1]*PLOT_FACTOR*1.15,s=list(heart_index.keys())[i],color = 'r', ha = 'center', va = 'center')
    else:
        plt.arrow(0,0,p_comp.components_[:2,i].T[0]*PLOT_FACTOR,p_comp.components_[:2,i].T[1]*PLOT_FACTOR,color='g',alpha=0.5)
        plt.text(p_comp.components_[:2,i].T[0]*PLOT_FACTOR*1.15,p_comp.components_[:2,i].T[1]*PLOT_FACTOR*1.15,s=list(heart_index.keys())[i],color = 'g', ha = 'center', va = 'center')        
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend(*sc.legend_elements(), title='Label');

### ICA

#### FMINST Data

In [ ]:
#Applying ICA on the FMINST Dataset

#Remembering that the number of features for FMINST are 784
dataset=X_fm_scaled
chosen_n_ica_d1=None

ica_size=range(1,250)
repeats=3
ideal_comps=None 

kurtosis=np.zeros(len(ica_size))
reconstruction_error=np.zeros(len(ica_size))

for i_ind,i in tqdm(enumerate(ica_size)):
    for r in [21*(1+i) for i in range(repeats)]:
        ica_comp=FastICA(n_components=i,random_state=r,max_iter=500)
        ica_comp.fit(dataset)
        kurtosis[i_ind]+=scipy.stats.kurtosis(np.abs(ica_comp.transform(dataset))).mean()/repeats

        #Getting error
        dataset_transformed=ica_comp.transform(dataset)
        dataset_reconstructed=ica_comp.inverse_transform(dataset_transformed)
        error=mean_squared_error(dataset,dataset_reconstructed)
        reconstruction_error[i_ind]+=error/3

#Plotting explained variance 
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('Mean Kurtosis & Number of ICA Components (FMINST)')
plt.plot(ica_size,kurtosis)
plt.scatter(ica_size, kurtosis, s=10, c='blue', marker='o', edgecolors='black')
plt.xlabel('N_components')
plt.ylabel('Mean Kurtosis')

#Plotting reconstruction error 
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('Reconstruction Error & Number of ICA Components (FMINST)')
plt.plot(ica_size,reconstruction_error)
plt.scatter(ica_size, reconstruction_error, s=10, c='blue', marker='o', edgecolors='black')
plt.axvline(ica_size[np.argmax(kurtosis)],color='red',linestyle='--',label=f'chosen Components')
plt.xlabel('N_components')
plt.ylabel('Reconstruction Error')
plt.legend()

##### Visualizing ICA Results for FMNIST

In [ ]:
#Plotting the images generated by ICA 
dataset=X_fm_scaled
chosen_n_ica_d1=66
ica_ideal_d1=FastICA(n_components=chosen_n_ica_d1,random_state=21,max_iter=500)
d1_ica=ica_ideal_d1.fit_transform(dataset)
titles=[f'ICA Component {i+1}' for i in range(chosen_n_ica_d1)]
plot_gallery(ica_ideal_d1.components_,titles, 28, 28,n_row=1,n_col=8)

correlation_matrix = np.corrcoef(d1_ica.T)

# Plot heatmap of new dataset 
plt.figure(figsize=(10, 5))
sns.heatmap(correlation_matrix, cmap='magma', fmt=".2f", cbar=True)
plt.title('Correlation Matrix of Independent Components (FMINST)')
plt.xlabel('Component Index')
plt.ylabel('Component Index')
plt.show()

correlation_matrix = np.corrcoef(dataset.T)

# Plot heatmap of old dataset 
plt.figure(figsize=(10, 5))
sns.heatmap(correlation_matrix, cmap='magma', fmt=".2f", cbar=True)
plt.title('Correlation Matrix of Original Features')
plt.xlabel('Feature Index')
plt.ylabel('Feature Index')
plt.show()

In [ ]:
#Plotting 
tsne_vis=TSNE(n_components=2,perplexity=45,n_jobs=-1)

#Transforming dataset 1 based on ideal 
dataset_transformed=tsne_vis.fit_transform(d1_ica)

fig, axes = plt.subplots(1, 2, figsize=(25, 5))

position_dict = {}

plt.suptitle('Visualizations K-MODES Clustering Results with Labels (FMINST)', fontsize=18)
LABS=[labels_og]

for lab in np.unique(labels_og):
    x, y = dataset_transformed[labels_og == lab][:, 0].mean(), dataset_transformed[labels_og == lab][:, 1].mean()
    position_dict[lab] = [x, y]

axis_int = 0
for ax, labels_ in zip(axes, LABS):
    artists = []

    for l in np.unique(labels_):
        # Choose color from a colormap
        color = plt.cm.viridis(l / len(np.unique(labels_)))
        ax.scatter(dataset_transformed[:, 0][labels_ == l], dataset_transformed[:, 1][labels_ == l],
                   label=l, zorder=1, color=color)

    # Overlay each image within the scatter plot
    for lab in np.unique(labels_og):
        # Positioning Images
        image_x = position_dict[lab][0]
        image_y = position_dict[lab][1]
        im = OffsetImage(X_fm_scaled[labels_og == lab][0].reshape(28, 28), cmap='gray')
        ab = AnnotationBbox(im, (image_x, image_y), xycoords='data', frameon=False)
        artists.append(ax.add_artist(ab))

    if axis_int == 0:
        ax.set_title(f'Actual')
        axis_int += 1
    else:
        ax.set_title(f'Clustered')

    ax.legend()

plt.tight_layout()
plt.show()

#### Heart Data

In [ ]:
#Applying ICA on the Heart Dataset

#Remembering that the number of features for FMINST are 784
dataset=X_heart_scaled
chosen_n_ica_d1=None

ica_size=range(1,21)
repeats=3
ideal_comps=None 

kurtosis=np.zeros(len(ica_size))
reconstruction_error=np.zeros(len(ica_size))

for i_ind,i in tqdm(enumerate(ica_size)):
    
    for r in [21*(1+i) for i in range(repeats)]:

        ica_comp=FastICA(n_components=i,random_state=r,max_iter=100000)
        ica_comp.fit(dataset)
        kurtosis[i_ind]+=scipy.stats.kurtosis(np.abs(ica_comp.transform(dataset))).mean()/repeats

        #Getting error
        dataset_transformed=ica_comp.transform(dataset)
        dataset_reconstructed=ica_comp.inverse_transform(dataset_transformed)
        error=mean_squared_error(dataset,dataset_reconstructed)
        reconstruction_error[i_ind]+=error/repeats

#Plotting explained variance 
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('Mean Kurtosis & Number of ICA Components (HEART)')
plt.plot(ica_size,kurtosis)
plt.scatter(ica_size, kurtosis, s=10, c='blue', marker='o', edgecolors='black')
plt.xlabel('N_components')
plt.ylabel('Mean Kurtosis')

#Plotting reconstruction error 
fig=plt.figure()
fig.set_size_inches(10,5)

plt.title('Reconstruction Error & Number of ICA Components (HEART)')
plt.plot(ica_size,reconstruction_error)
plt.scatter(ica_size, reconstruction_error, s=10, c='blue', marker='o', edgecolors='black')
plt.axvline(ica_size[np.argmax(kurtosis)],color='red',linestyle='--',label=f'chosen Components')
plt.xlabel('N_components')
plt.ylabel('Reconstruction Error')
plt.legend()

##### Visualizing ICA Results for Heart

In [ ]:
dataset=X_heart_scaled
#chosen_n_ica_d2=ica_size[np.argmax(kurtosis)]
ica_comp=FastICA(n_components=chosen_n_ica_d2,random_state=r,max_iter=100000)
d2_ica=ica_comp.fit_transform(dataset)

correlation_matrix = np.corrcoef(d2_ica.T)

# Plot heatmap of new dataset 
plt.figure(figsize=(10, 5))
sns.heatmap(correlation_matrix, cmap='magma', fmt=".2f", cbar=True)
plt.title('Correlation Matrix of Independent Components (HEART)')
plt.xlabel('Component Index')
plt.ylabel('Component Index')
plt.show()

correlation_matrix = np.corrcoef(dataset.T)

# Plot heatmap of old dataset 
plt.figure(figsize=(10, 5))
sns.heatmap(correlation_matrix, cmap='magma', fmt=".2f", cbar=True)
plt.title('Correlation Matrix of Original Features (HEART)')
plt.xlabel('Feature Index')
plt.ylabel('Feature Index')
plt.show()

#Projecting the dataset into the first 2 dimensions
fig,axes=plt.subplots(1,2)
fig.set_size_inches(20,5)
plt.suptitle('Visualizing ICA for Selected Components & Features (HEART)')

SELECTION=[[5,6],[6,7]]

for ii,ica_select in enumerate(SELECTION):
    PLOT_FACTOR=3
    ICA_COMP_1=ica_select[0]
    ICA_COMP_2=ica_select[1]

    filtered_components=ica_comp.components_[[ICA_COMP_1,ICA_COMP_2]]
    dataset_2d=np.dot(dataset,filtered_components.T)


    sc=axes[ii].scatter(dataset_2d[:,0],dataset_2d[:,1],c=labels,cmap='plasma',s=5)
    most_important=np.argsort(np.abs(filtered_components).sum(axis=0))[::-1][:4].ravel()

    for i in range(dataset.shape[-1]):
        if i in most_important:
            axes[ii].arrow(0,0,filtered_components[:,i].T[0]*PLOT_FACTOR,filtered_components[:,i].T[1]*PLOT_FACTOR,color='r',alpha=0.5)
            axes[ii].text(filtered_components[:,i].T[0]*PLOT_FACTOR*1.15,filtered_components[:,i].T[1]*PLOT_FACTOR*1.15,s=list(heart_index.keys())[i],color = 'r', ha = 'center', va = 'center')
        else:
            pass
            #plt.arrow(0,0,ica_comp.components_[:2,i].T[0]*PLOT_FACTOR,ica_comp.components_[:2,i].T[1]*PLOT_FACTOR,color='g',alpha=0.5)
            #plt.text(ica_comp.components_[:2,i].T[0]*PLOT_FACTOR*1.15,ica_comp.components_[:2,i].T[1]*PLOT_FACTOR*1.15,s=list(heart_index.keys())[i],color = 'g', ha = 'center', va = 'center')        
    axes[ii].set_xlabel(f'ICA Component {ICA_COMP_1}')
    axes[ii].set_ylabel(f'ICA Component {ICA_COMP_2}')
    axes[ii].legend(*sc.legend_elements(), title='Label');

### Randomized Projection

#### FMINST Data

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.random_projection import GaussianRandomProjection
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# Sample data and calculations
dataset=X_fm_scaled
rgp_size=range(1,1000)
repeats = 3
original_distance = scipy.spatial.distance.pdist(dataset)
reconstruction_error = np.zeros(len(rgp_size))
reconstruction_vals = np.zeros(shape=(len(rgp_size), repeats))
pairwise_distance_error = np.zeros(len(rgp_size))
pairwise_distance_errors= np.zeros(shape=(len(rgp_size), repeats))

for i_ind, i in tqdm(enumerate(rgp_size)):
    for ix, r in enumerate([21 * (1 + i) for i in range(repeats)]):
        rgp_comp = GaussianRandomProjection(n_components=i, random_state=r)
        rgp_comp.fit(dataset)

        dataset_transformed = rgp_comp.transform(dataset)
        dataset_reconstructed = rgp_comp.inverse_transform(dataset_transformed)
        error = mean_squared_error(dataset, dataset_reconstructed)
        reconstruction_error[i_ind] += error / 3
        reconstruction_vals[i_ind, ix] = error

        pairwise_distance_error[i_ind]+=((original_distance-scipy.spatial.distance.pdist(dataset_transformed))**2).sum()/3
        pairwise_distance_errors[i_ind, ix]=((original_distance-scipy.spatial.distance.pdist(dataset_transformed))**2).sum()

# Plotting reconstruction error with filled space between error bars
fig = plt.figure()
fig.set_size_inches(10, 5)

plt.title('Reconstruction Error & Number of Random Gaussian Projection Components (FMINST)')
plt.errorbar(rgp_size, reconstruction_error, yerr=np.std(reconstruction_vals, axis=1), fmt='o', capsize=5,color='orange')
plt.fill_between(rgp_size, reconstruction_error - np.std(reconstruction_vals, axis=1), reconstruction_error + np.std(reconstruction_vals, axis=1), alpha=0.2,color='orange',label='error')
plt.plot(rgp_size,reconstruction_error,color='orange')
plt.xlabel('N_components')
plt.ylabel('Reconstruction Error')
plt.legend()
plt.xticks(rgp_size);

# Plotting Pairwise Distance with filled space between error bars
fig = plt.figure()
fig.set_size_inches(10, 5)

plt.title('Pairwise Distance Error & Number of Random Gaussian Projection Components (FMINST)')
plt.errorbar(rgp_size, pairwise_distance_error, yerr=np.std(pairwise_distance_errors, axis=1), fmt='o', capsize=5,color='orange')
plt.fill_between(rgp_size, pairwise_distance_error - np.std(pairwise_distance_errors, axis=1), pairwise_distance_error + np.std(pairwise_distance_errors, axis=1), alpha=0.2, color='orange',label='error')
plt.plot(rgp_size,pairwise_distance_error, color='orange')
plt.xlabel('N_components')
plt.ylabel('Pairwise Distance Error')
plt.xticks(rgp_size);
plt.legend()

plt.show()

In [ ]:
#Using the Distance Metric 
rgp_comp=GaussianRandomProjection(n_components=johnson_lindenstrauss_min_dim(X_heart_scaled.shape[0],eps=0.323),random_state=r)
dataset_transformed=rgp_comp.fit_transform(dataset)

dataset_reconstructed=rgp_comp.inverse_transform(dataset_transformed)
error=mean_squared_error(dataset,dataset_reconstructed)
reconstruction_error=error

#Calculating Pairwise Distance
pairwise_distance_error=((original_distance-scipy.spatial.distance.pdist(dataset_transformed))**2).sum()/3

##### Visualizing Random Projection Results for FMINST

In [ ]:
#For selection random projection value, we use a reconstruction error of 0.15 to select the number of freatures for random projection. We chose this as this was the error that was produced by our ICA and PCA reduction techniques after selection.
dataset=X_fm_scaled
ERROR_CHOICE_D1=0.175
#chosen_n_rgp_d1=np.argwhere(reconstruction_error<ERROR_CHOICE_D1)[0][0]
chosen_n_rgp_d1=666

#Creating the projection
d1_rgp_ideal=GaussianRandomProjection(n_components=chosen_n_rgp_d1,random_state=21)
d1_rgp_projection=d1_rgp_ideal.fit_transform(dataset)

#Visualizing projection with distance 
fig=plt.figure()
fig.set_size_inches(10,5)
pairwise_distances_original = scipy.spatial.distance.pdist(dataset)
pairwise_distances_rp = scipy.spatial.distance.pdist(d1_rgp_projection)

plt.scatter(pairwise_distances_original, pairwise_distances_rp,s=5,color='orange')
plt.plot((0,max(pairwise_distances_rp)),(0,max(pairwise_distances_rp)),color='red',label='Ideal',linestyle='--')
plt.xlabel('Pairwise Distances Before Projection')
plt.ylabel('Pairwise Distances After Projection')
plt.title('Pairwise Distances Before and After Random Projection (FMINST)')
plt.legend()
plt.show()

#Visualizing with Projection
fig=plt.figure()
fig.set_size_inches(10,5)
tsne = TSNE(n_components=2)
X_tsne_proj = tsne.fit_transform(d1_rgp_projection)
X_tsne_original=tsne.fit_transform(dataset)

plt.scatter(X_tsne_proj[:, 0], X_tsne_proj[:, 1],color='orange',s=5,label='transformed')
plt.scatter(X_tsne_original[:, 0], X_tsne_original[:, 1],color='blue',s=5,label='original')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('t-SNE Visualization of Random Projection (FMINST)')
plt.legend()
plt.show()

##Plotting the images generated by Random Projection 
titles=[f'Rand Proj Comp {i+1}' for i in range(chosen_n_rgp_d1)]
plot_gallery(d1_rgp_ideal.components_,titles, 28, 28)


#### Heart Data

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.random_projection import GaussianRandomProjection
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# Sample data and calculations
dataset = X_heart_scaled
rgp_size = range(1, 21)
repeats = 3
original_distance = scipy.spatial.distance.pdist(dataset)
reconstruction_error = np.zeros(len(rgp_size))
reconstruction_vals = np.zeros(shape=(len(rgp_size), repeats))
pairwise_distance_error = np.zeros(len(rgp_size))
pairwise_distance_errors= np.zeros(shape=(len(rgp_size), repeats))

for i_ind, i in tqdm(enumerate(rgp_size)):
    for ix, r in enumerate([21 * (1 + i) for i in range(repeats)]):
        rgp_comp = GaussianRandomProjection(n_components=i, random_state=r)
        rgp_comp.fit(dataset)

        dataset_transformed = rgp_comp.transform(dataset)
        dataset_reconstructed = rgp_comp.inverse_transform(dataset_transformed)
        error = mean_squared_error(dataset, dataset_reconstructed)
        reconstruction_error[i_ind] += error / 3
        reconstruction_vals[i_ind, ix] = error

        pairwise_distance_error[i_ind]+=((original_distance-scipy.spatial.distance.pdist(dataset_transformed))**2).sum()/3
        pairwise_distance_errors[i_ind, ix]=((original_distance-scipy.spatial.distance.pdist(dataset_transformed))**2).sum()

# Plotting reconstruction error with filled space between error bars
fig = plt.figure()
fig.set_size_inches(10, 5)

plt.title('Reconstruction Error & Number of Random Gaussian Projection Components (HEART)')
plt.errorbar(rgp_size, reconstruction_error, yerr=np.std(reconstruction_vals, axis=1), fmt='o', capsize=5,color='orange')
plt.fill_between(rgp_size, reconstruction_error - np.std(reconstruction_vals, axis=1), reconstruction_error + np.std(reconstruction_vals, axis=1), alpha=0.2,color='orange',label='error')
plt.plot(rgp_size,reconstruction_error,color='orange')
plt.xlabel('N_components')
plt.ylabel('Reconstruction Error')
plt.legend()
plt.xticks(rgp_size);

# Plotting Pairwise Distance with filled space between error bars
fig = plt.figure()
fig.set_size_inches(10, 5)

plt.title('Pairwise Distance Error & Number of Random Gaussian Projection Components (HEART)')
plt.errorbar(rgp_size, pairwise_distance_error, yerr=np.std(pairwise_distance_errors, axis=1), fmt='o', capsize=5,color='orange')
plt.fill_between(rgp_size, pairwise_distance_error - np.std(pairwise_distance_errors, axis=1), pairwise_distance_error + np.std(pairwise_distance_errors, axis=1), alpha=0.2, color='orange',label='error')
plt.plot(rgp_size,pairwise_distance_error, color='orange')
plt.xlabel('N_components')
plt.ylabel('Pairwise Distance Error')
plt.xticks(rgp_size);
plt.legend()

plt.show()

##### Visualizing Random Projection Results for Heart

In [ ]:
#For selection random projection value, we use a reconstruction error of 0.15 to select the number of freatures for random projection. We chose this as this was the error that was produced by our ICA and PCA reduction techniques after selection.
dataset=X_heart_scaled
ERROR_CHOICE_D2=0.15
chosen_n_rgp_d2=np.argwhere(reconstruction_error<ERROR_CHOICE_D2)[0][0]

#Creating the projection
d2_rgp_ideal=GaussianRandomProjection(n_components=chosen_n_rgp_d2,random_state=21)
d2_rgp_projection=d2_rgp_ideal.fit_transform(dataset)

#Visualizing projection with distance 
fig=plt.figure()
fig.set_size_inches(10,5)
pairwise_distances_original = scipy.spatial.distance.pdist(dataset)
pairwise_distances_rp = scipy.spatial.distance.pdist(d2_rgp_projection)

plt.scatter(pairwise_distances_original, pairwise_distances_rp,s=5,color='orange')
plt.plot((0,max(pairwise_distances_rp)),(0,max(pairwise_distances_rp)),color='red',label='Ideal',linestyle='--')
plt.xlabel('Pairwise Distances Before Projection')
plt.ylabel('Pairwise Distances After Projection')
plt.title('Pairwise Distances Before and After Random Projection (HEART)')
plt.legend()
plt.show()

#Visualizing with Projection
fig=plt.figure()
fig.set_size_inches(10,5)
tsne = TSNE(n_components=2)
X_tsne_proj = tsne.fit_transform(d2_rgp_projection)
X_tsne_original=tsne.fit_transform(dataset)

plt.scatter(X_tsne_proj[:, 0], X_tsne_proj[:, 1],color='orange',s=5,label='transformed')
plt.scatter(X_tsne_original[:, 0], X_tsne_original[:, 1],color='blue',s=5,label='original')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('t-SNE Visualization of Random Projection (HEART)')
plt.legend()
plt.show()

### Manifold

#### FMINST Data

In [ ]:
#Applying UMAP Manifold Learning on the FMINST Dataset

#Remembering that the number of features for FMINST are 784

dataset=X_fm_scaled
chosen_n_umap_d1=None
chosen_neighbors_1=None

n_comps=range(2,100)
num_neighbors=[45]
repeats=1
ideal_comps=None 

trustworthiness_scores=np.zeros(shape=(len(n_comps),len(num_neighbors)))
reconstruction_errors=np.zeros(shape=(len(n_comps),len(num_neighbors)))

for i,n in tqdm(enumerate(n_comps)):

    for ii,neighbors in enumerate(num_neighbors):

        umap_comp=umap.UMAP(n_components=n,n_neighbors=neighbors,n_jobs=-1)
        dataset_transformed=umap_comp.fit_transform(dataset)

        trustworthiness_scores[i,ii]=trustworthiness(dataset,dataset_transformed,n_neighbors=neighbors)
        reconstruction_errors[i,ii]=mean_squared_error(dataset,umap_comp.inverse_transform(dataset_transformed))

fig=plt.figure()
fig.set_size_inches(10,5)
plt.title('Trustworthiness Score, Num Neighbors & N_Components (FMINST)');
for ii,neighbors in enumerate(num_neighbors):
    plt.scatter(n_comps,trustworthiness_scores[:,ii],label=f'Num Neighbours {neighbors}')
    plt.plot(n_comps,trustworthiness_scores[:,ii])
    plt.legend()
plt.xlabel('N_Components')
plt.ylabel('Trustworthiness Score')

fig=plt.figure()
fig.set_size_inches(10,5)
plt.title('Reconstruction Error, Num Neighbors & N_Components (FMINST)');
for ii,neighbors in enumerate(num_neighbors):
    plt.scatter(n_comps,reconstruction_errors[:,ii],label=f'Num Neighbours {neighbors}')
    plt.plot(n_comps,reconstruction_errors[:,ii])
    plt.legend()
plt.xlabel('N_Components')
plt.ylabel('Reconstruction Error')

In [ ]:
#Applying ISOMAP Manifold Learning on the FMINST Dataset

dataset=X_fm_scaled
chosen_n_iso_map_d1=None
chosen_neighbors_d1=None

n_comps=range(2,100)
num_neighbors=[30,45,60]
repeats=3
ideal_comps=None 

trustworthiness_scores=np.zeros(shape=(len(n_comps),len(num_neighbors)))
reconstruction_errors=np.zeros(shape=(len(n_comps),len(num_neighbors)))

for i,n in tqdm(enumerate(n_comps)):

    for ii,neighbors in enumerate(num_neighbors):

        iso_map_comp=Isomap(n_components=n,n_neighbors=neighbors,n_jobs=-1)
        dataset_transformed=iso_map_comp.fit_transform(dataset)

        trustworthiness_scores[i,ii]=trustworthiness(dataset,dataset_transformed,n_neighbors=neighbors,)
        reconstruction_errors[i,ii]=iso_map_comp.reconstruction_error()

fig=plt.figure()
fig.set_size_inches(10,5)
plt.title('Trustworthiness Score, Num Neighbors & N_Components (FMINST)');
for ii,neighbors in enumerate(num_neighbors):
    plt.scatter(n_comps,trustworthiness_scores[:,ii],label=f'Num Neighbours {neighbors}')
    plt.plot(n_comps,trustworthiness_scores[:,ii])
    plt.legend()
plt.xlabel('N_Components')
plt.ylabel('Trustworthiness Score')

fig=plt.figure()
fig.set_size_inches(10,5)
plt.title('Reconstruction Error, Num Neighbors & N_Components (FMINST)');
for ii,neighbors in enumerate(num_neighbors):
    plt.scatter(n_comps,reconstruction_errors[:,ii],label=f'Num Neighbours {neighbors}')
    plt.plot(n_comps,reconstruction_errors[:,ii])
    plt.legend()
plt.xlabel('N_Components')
plt.ylabel('Reconstruction Error')

In [ ]:
#Creating ideal UMAP manifold 
chosen_n_umap_d1=11
chosen_neighbors_d1=45
dataset=X_fm_scaled
labels=Y_fm

#iso_map_d1=umap.UMAP(n_components=chosen_n_iso_map_d1,n_neighbors=chosen_neighbors_d1,n_jobs=-1)
iso_map_d1=umap.UMAP(n_components=chosen_n_umap_d1,n_neighbors=chosen_neighbors_d1,n_jobs=-1)

#Transforming dataset 1 based on ideal 
dataset_transformed=iso_map_d1.fit_transform(dataset)

#Visualizing transformed data
fig=plt.figure()
fig, axes = plt.subplots(1,3,figsize=(25, 5))
position_dict={}

plt.suptitle('Visualizations of Select UMAP Components (FMINST)',fontsize=18)

SELECTION=[[0,1],[0,2],[9,10]]

for ax,dims in zip(axes,SELECTION):
    artists = []
    dim_1=dims[0]
    dim_2=dims[1]

    for lab in np.unique(Y_fm):
        x,y=dataset_transformed[labels==lab][:,dim_1].mean(),dataset_transformed[labels==lab][:,dim_2].mean()
        position_dict[lab]=[x,y]

    sc=ax.scatter(dataset_transformed[:,dim_1],dataset_transformed[:,dim_2],c=Y_fm,cmap='viridis',zorder=1)
    # Overlay each image within the scatter plot
    for lab in np.unique(Y_fm):
        # Positioning Images
        image_x = position_dict[lab][0]
        image_y = position_dict[lab][1]
        im=OffsetImage(dataset[labels==lab][0].reshape(28,28),cmap='gray')
        ab = AnnotationBbox(im, (image_x, image_y), xycoords='data', frameon=False)
        artists.append(ax.add_artist(ab))
        ax.set_title(f'Dimension {dim_1} & Dimension {dim_2}')

    ax.legend(*sc.legend_elements(), title='Label')
plt.tight_layout()

In [ ]:
#Creating ideal ISOmap manifold 
chosen_n_iso_map_d1=20
chosen_neighbors_d1=45
dataset=X_fm_scaled
labels=Y_fm

#iso_map_d1=umap.UMAP(n_components=chosen_n_iso_map_d1,n_neighbors=chosen_neighbors_d1,n_jobs=-1)
iso_map_d1=Isomap(n_components=chosen_n_iso_map_d1,n_neighbors=chosen_neighbors_d1,n_jobs=-1)

#Transforming dataset 1 based on ideal 
dataset_transformed=iso_map_d1.fit_transform(dataset)

#Visualizing transformed data
fig=plt.figure()
fig, axes = plt.subplots(1,3,figsize=(25, 5))
position_dict={}

plt.suptitle('Visualizations of Select ISOMAP Components (FMINST)',fontsize=18)

SELECTION=[[0,1],[0,2],[9,10]]

for ax,dims in zip(axes,SELECTION):
    artists = []
    dim_1=dims[0]
    dim_2=dims[1]

    for lab in np.unique(Y_fm):
        x,y=dataset_transformed[labels==lab][:,dim_1].mean(),dataset_transformed[labels==lab][:,dim_2].mean()
        position_dict[lab]=[x,y]

    sc=ax.scatter(dataset_transformed[:,dim_1],dataset_transformed[:,dim_2],c=Y_fm,cmap='viridis',zorder=1)
    # Overlay each image within the scatter plot
    for lab in np.unique(Y_fm):
        # Positioning Images
        image_x = position_dict[lab][0]
        image_y = position_dict[lab][1]
        im=OffsetImage(dataset[labels==lab][0].reshape(28,28),cmap='gray')
        ab = AnnotationBbox(im, (image_x, image_y), xycoords='data', frameon=False)
        artists.append(ax.add_artist(ab))
        ax.set_title(f'Dimension {dim_1} & Dimension {dim_2}')

    ax.legend(*sc.legend_elements(), title='Label')
plt.tight_layout()

#### Heart Data

In [ ]:
#Applying UMAP Manifold Learning on the Heart Dataset

dataset=X_heart_scaled
chosen_n_umap_d1=None
chosen_neighbors_1=None

n_comps=range(1,20)
num_neighbors=[30,45,60]
repeats=3
ideal_comps=None 

trustworthiness_scores=np.zeros(shape=(len(n_comps),len(num_neighbors)))
reconstruction_errors=np.zeros(shape=(len(n_comps),len(num_neighbors)))

for i,n in tqdm(enumerate(n_comps)):

    for ii,neighbors in enumerate(num_neighbors):

        umap_comp=umap.UMAP(n_components=n,n_neighbors=neighbors,n_jobs=-1)
        dataset_transformed=umap_comp.fit_transform(dataset)

        trustworthiness_scores[i,ii]=trustworthiness(dataset,dataset_transformed,n_neighbors=neighbors)
        #reconstruction_errors[i,ii]=mean_squared_error(dataset,umap_comp.inverse_transform(dataset_transformed))

fig=plt.figure()
fig.set_size_inches(10,5)
plt.title('Trustworthiness Score, Num Neighbors & N_Components (HEART)');
for ii,neighbors in enumerate(num_neighbors):
    plt.scatter(n_comps,trustworthiness_scores[:,ii],label=f'Num Neighbours {neighbors}')
    plt.plot(n_comps,trustworthiness_scores[:,ii])
    plt.legend()
plt.xlabel('N_Components')
plt.ylabel('Trustworthiness Score')

# fig=plt.figure()
# fig.set_size_inches(10,5)
# plt.title('Reconstruction Error, Num Neighbors & N_Components (HEART)');
# for ii,neighbors in enumerate(num_neighbors):
#     plt.scatter(n_comps,reconstruction_errors[:,ii],label=f'Num Neighbours {neighbors}')
#     plt.plot(n_comps,reconstruction_errors[:,ii])
#     plt.legend()
# plt.xlabel('N_Components')
# plt.ylabel('Reconstruction Error')

In [ ]:
#Applying ISOMAP Manifold Learning on the Heart Dataset

dataset=X_heart_scaled
chosen_n_umap_d2=None
chosen_neighbors_d2=None

n_comps=range(2,20)
num_neighbors=[30,45,60]
repeats=3
ideal_comps=None 

trustworthiness_scores=np.zeros(shape=(len(n_comps),len(num_neighbors)))
reconstruction_errors=np.zeros(shape=(len(n_comps),len(num_neighbors)))

for i,n in tqdm(enumerate(n_comps)):

    for ii,neighbors in enumerate(num_neighbors):

        iso_map_comp=Isomap(n_components=n,n_neighbors=neighbors,n_jobs=-1,p=1)
        dataset_transformed=iso_map_comp.fit_transform(dataset)

        trustworthiness_scores[i,ii]=trustworthiness(dataset,dataset_transformed,n_neighbors=neighbors)
        reconstruction_errors[i,ii]=iso_map_comp.reconstruction_error()

fig=plt.figure()
fig.set_size_inches(10,5)
plt.title('Trustworthiness Score, Num Neighbors & N_Components (HEART)');
for ii,neighbors in enumerate(num_neighbors):
    plt.scatter(n_comps,trustworthiness_scores[:,ii],label=f'Num Neighbours {neighbors}')
    plt.plot(n_comps,trustworthiness_scores[:,ii])
    plt.legend()
plt.xlabel('N_Components')
plt.ylabel('Trustworthiness Score')

fig=plt.figure()
fig.set_size_inches(10,5)
plt.title('Reconstruction Error, Num Neighbors & N_Components (HEART)');
for ii,neighbors in enumerate(num_neighbors):
    plt.scatter(n_comps,reconstruction_errors[:,ii],label=f'Num Neighbours {neighbors}')
    plt.plot(n_comps,reconstruction_errors[:,ii])
    plt.legend()
plt.xlabel('N_Components')
plt.ylabel('Reconstruction Error')

In [ ]:
#Applying ISOMAP Manifold Learning on the Heart Dataset

dataset=X_heart_scaled
chosen_n_umap_d2=None
chosen_neighbors_d2=None

n_comps=range(2,20)
P=[1,2,3]
repeats=3
ideal_comps=None 

trustworthiness_scores=np.zeros(shape=(len(n_comps),len(P)))
reconstruction_errors=np.zeros(shape=(len(n_comps),len(P)))

for i,n in tqdm(enumerate(n_comps)):

    for ii,p in enumerate(P):

        iso_map_comp=Isomap(n_components=n,n_neighbors=45,n_jobs=-1,p=p)
        dataset_transformed=iso_map_comp.fit_transform(dataset)

        trustworthiness_scores[i,ii]=trustworthiness(dataset,dataset_transformed,n_neighbors=neighbors)
        reconstruction_errors[i,ii]=iso_map_comp.reconstruction_error()

fig=plt.figure()
fig.set_size_inches(10,5)
plt.title('Trustworthiness Score, Num Neighbors & N_Components (HEART)');
for ii,p in enumerate(P):
    plt.scatter(n_comps,trustworthiness_scores[:,ii],label=f'Distance Measure {p}')
    plt.plot(n_comps,trustworthiness_scores[:,ii])
    plt.legend()
plt.xlabel('N_Components')
plt.ylabel('Trustworthiness Score')

fig=plt.figure()
fig.set_size_inches(10,5)
plt.title('Reconstruction Error, Num Neighbors & N_Components (HEART)');
for ii,p in enumerate(P):
    plt.scatter(n_comps,reconstruction_errors[:,ii],label=f'Distance Measure {p}')
    plt.plot(n_comps,reconstruction_errors[:,ii])
    plt.legend()
plt.xlabel('N_Components')
plt.ylabel('Reconstruction Error')

In [ ]:
#Different values of p
dataset=X_heart_scaled
chosen_n_umap_d2=None
chosen_neighbors_d2=None

n_comps=range(2,20)
num_neighbors=[30,45,60]
repeats=3
ideal_comps=None 

trustworthiness_scores=np.zeros(shape=(len(n_comps),len(num_neighbors)))
reconstruction_errors=np.zeros(shape=(len(n_comps),len(num_neighbors)))

for i,n in tqdm(enumerate(n_comps)):

    for ii,neighbors in enumerate(num_neighbors):

        iso_map_comp=Isomap(n_components=n,n_neighbors=neighbors,n_jobs=-1)
        dataset_transformed=iso_map_comp.fit_transform(dataset)

        trustworthiness_scores[i,ii]=trustworthiness(dataset,dataset_transformed,n_neighbors=neighbors)
        reconstruction_errors[i,ii]=iso_map_comp.reconstruction_error()

fig=plt.figure()
fig.set_size_inches(10,5)
plt.title('Trustworthiness Score, Num Neighbors & N_Components (HEART)');
for ii,neighbors in enumerate(num_neighbors):
    plt.scatter(n_comps,trustworthiness_scores[:,ii],label=f'Num Neighbours {neighbors}')
    plt.plot(n_comps,trustworthiness_scores[:,ii])
    plt.legend()
plt.xlabel('N_Components')
plt.ylabel('Trustworthiness Score')

fig=plt.figure()
fig.set_size_inches(10,5)
plt.title('Reconstruction Error, Num Neighbors & N_Components (HEART)');
for ii,neighbors in enumerate(num_neighbors):
    plt.scatter(n_comps,reconstruction_errors[:,ii],label=f'Num Neighbours {neighbors}')
    plt.plot(n_comps,reconstruction_errors[:,ii])
    plt.legend()
plt.xlabel('N_Components')
plt.ylabel('Reconstruction Error')

In [ ]:
#Creating ideal UMAP manifold 
chosen_n_umap_d2=5
chosen_neighbors_d2=45
dataset=X_heart_scaled
labels=Y_heart

#iso_map_d1=umap.UMAP(n_components=chosen_n_iso_map_d1,n_neighbors=chosen_neighbors_d1,n_jobs=-1)
umap_d2=umap.UMAP(n_components=chosen_n_umap_d2,n_neighbors=chosen_neighbors_d2,n_jobs=-1)

#Transforming dataset 1 based on ideal 
dataset_transformed=umap_d2.fit_transform(dataset)

#Visualizing transformed data
fig=plt.figure()
fig, axes = plt.subplots(1,3,figsize=(25, 5))
position_dict={}

plt.suptitle('Visualizations of Select UMAP Components (HEART)',fontsize=18)

SELECTION=[[0,1],[0,2],[3,4]]

for ax,dims in zip(axes,SELECTION):
    artists = []
    dim_1=dims[0]
    dim_2=dims[1]

    # for lab in np.unique(Y_fm):
    #     x,y=dataset_transformed[labels==lab][:,dim_1].mean(),dataset_transformed[labels==lab][:,dim_2].mean()
    #     position_dict[lab]=[x,y]

    sc=ax.scatter(dataset_transformed[:,dim_1],dataset_transformed[:,dim_2],c=labels,cmap='viridis',zorder=1)
    # Overlay each image within the scatter plot
    # for lab in np.unique(Y_fm):
    #     # Positioning Images
    #     image_x = position_dict[lab][0]
    #     image_y = position_dict[lab][1]
    #     im=OffsetImage(dataset[labels==lab][0].reshape(28,28),cmap='gray')
    #     ab = AnnotationBbox(im, (image_x, image_y), xycoords='data', frameon=False)
    #     artists.append(ax.add_artist(ab))
    ax.set_title(f'Dimension {dim_1} & Dimension {dim_2}')

    ax.legend(*sc.legend_elements(), title='Label')
plt.tight_layout()

In [ ]:
#Creating ideal ISOMAP manifold 
chosen_n_iso_map_d2=13
chosen_neighbors_d2=45
dataset=X_heart_scaled
labels=Y_heart

#iso_map_d1=umap.UMAP(n_components=chosen_n_iso_map_d1,n_neighbors=chosen_neighbors_d1,n_jobs=-1)
iso_map_d2=Isomap(n_components=chosen_n_iso_map_d2,n_neighbors=chosen_neighbors_d2,n_jobs=-1)

#Transforming dataset 1 based on ideal 
dataset_transformed=iso_map_d2.fit_transform(dataset)

#Visualizing transformed data
fig=plt.figure()
fig, axes = plt.subplots(1,3,figsize=(25, 5))
position_dict={}

plt.suptitle('Visualizations of Select ISOMAP Components (HEART)',fontsize=18)

SELECTION=[[0,1],[0,2],[11,12]]

for ax,dims in zip(axes,SELECTION):
    artists = []
    dim_1=dims[0]
    dim_2=dims[1]

    # for lab in np.unique(Y_fm):
    #     x,y=dataset_transformed[labels==lab][:,dim_1].mean(),dataset_transformed[labels==lab][:,dim_2].mean()
    #     position_dict[lab]=[x,y]

    sc=ax.scatter(dataset_transformed[:,dim_1],dataset_transformed[:,dim_2],c=labels,cmap='viridis',zorder=1)
    # Overlay each image within the scatter plot
    # for lab in np.unique(Y_fm):
    #     # Positioning Images
    #     image_x = position_dict[lab][0]
    #     image_y = position_dict[lab][1]
    #     im=OffsetImage(dataset[labels==lab][0].reshape(28,28),cmap='gray')
    #     ab = AnnotationBbox(im, (image_x, image_y), xycoords='data', frameon=False)
    #     artists.append(ax.add_artist(ab))
    ax.set_title(f'Dimension {dim_1} & Dimension {dim_2}')

    ax.legend(*sc.legend_elements(), title='Label')
plt.tight_layout()